In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import time_uuid
from uuid import UUID
from util.util import *
from util.dbFactory import DatabaseFactory


In [12]:
spark = create_spark_session()

In [13]:
data = spark.read.format("org.apache.spark.sql.cassandra").options(table = 'tracking',keyspace = 'logs').load()

In [14]:
def process_data(data):
    # UDF chuyển đổi từ TimeUUID sang datetime string
    def to_datetime_str(x):
        return time_uuid.TimeUUID(bytes=UUID(x).bytes).get_datetime().strftime('%Y-%m-%d %H:%M:%S')
    
    # Đăng ký UDF với Spark
    to_datetime_str_udf = udf(to_datetime_str, StringType())
    
    # Tạo cột mới 'ts' với giá trị datetime chuyển đổi từ 'create_time'
    result = data.withColumn('ts', to_datetime_str_udf('create_time'))
    
    # Lựa chọn các cột cần thiết
    # result = result.select('create_time', 'ts', 'job_id', 'custom_track', 'bid', 'campaign_id', 'group_id', 'publisher_id')
    
    return result

data = process_data(data)

In [17]:
data = data.cache()

In [16]:
data.show(5)

+--------------------+----+----------+-----------+---+------------+-----+--------------------+---------------+--------------------+---+--------+----+------+----+------------+--------------------+---------+-------------------+----+--------------------+-------------------+------------+-----------+----------+----------+--------+---+--------+
|         create_time| bid|        bn|campaign_id| cd|custom_track|   de|                  dl|             dt|                  ed| ev|group_id|  id|job_id|  md|publisher_id|                  rl|       sr|                 ts|  tz|                  ua|                uid|utm_campaign|utm_content|utm_medium|utm_source|utm_term|  v|      vp|
+--------------------+----+----------+-----------+---+------------+-----+--------------------+---------------+--------------------+---+--------+----+------+----+------------+--------------------+---------+-------------------+----+--------------------+-------------------+------------+-----------+----------+----------+

In [18]:

# Kết nối đến Cassandra
factory = DatabaseFactory()
session = factory.create_connect('cassandra')

# Xóa toàn bộ dữ liệu trong bảng
session.execute('TRUNCATE tracking')


Traceback (most recent call last):
  File "f:\ProjectDataPipeline\.venv\lib\site-packages\cassandra\cluster.py", line 3577, in _reconnect_internal
    return self._try_connect(host)
  File "f:\ProjectDataPipeline\.venv\lib\site-packages\cassandra\cluster.py", line 3599, in _try_connect
    connection = self._cluster.connection_factory(host.endpoint, is_control_connection=True)
  File "f:\ProjectDataPipeline\.venv\lib\site-packages\cassandra\cluster.py", line 1670, in connection_factory
    return self.connection_class.factory(endpoint, self.connect_timeout, *args, **kwargs)
  File "f:\ProjectDataPipeline\.venv\lib\site-packages\cassandra\connection.py", line 846, in factory
    conn = cls(endpoint, *args, **kwargs)
  File "f:\ProjectDataPipeline\.venv\lib\site-packages\cassandra\io\asyncorereactor.py", line 353, in __init__
    self._connect_socket()
  File "f:\ProjectDataPipeline\.venv\lib\site-packages\cassandra\connection.py", line 951, in _connect_socket
    raise socket.error(sock

In [19]:
data.write \
    .format("org.apache.spark.sql.cassandra") \
    .options(table="tracking", keyspace="logs") \
    .mode("append") \
    .save()